<a href="https://colab.research.google.com/github/faezeh-lbf/Probing-Persian-Language-Models/blob/main/Comparison_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/datasets.git
from IPython.display import clear_output 
!pip install -q sentencepiece
# !pip install ipywidgets
# !pip install bertviz
clear_output()

In [ ]:
!pip install tqdm --upgrade


## Loading the dataset

In [ ]:
from datasets import load_dataset, list_datasets, load_metric


BATCH_SIZE = 32
batch_size = BATCH_SIZE
# OPTIONS_ORDER = ['بزرگتر', 'کوچکتر']


# size
dataset_name = '/content/age_comp_train'
# datasets = load_dataset('json', data_files='/content/size_comp_train.jsonl').shuffle()
test_dataset = load_dataset('json', data_files='/content/age_comp_dev_nolang.jsonl').shuffle()['train']
# print(test_dataset)
# datasets = load_dataset('json', data_files='/content/breakable_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/breakable_dev.jsonl').shuffle()

# # num
# datasets = load_dataset('json', data_files='/content/ant-syn_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/ant-syn_dev.jsonl').shuffle()

# age
# datasets = load_dataset('json', data_files='/content/age_comp_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/age_comp_dev.jsonl').shuffle()

# datasets = datasets['train'].train_test_split(test_size=0.1)
# datasets['validation'] = datasets['test']
# datasets['test'] = dataset_test['train']

In [ ]:
# nolang datasets

from datasets import load_dataset, list_datasets, load_metric


BATCH_SIZE = 32
batch_size = BATCH_SIZE
# size
# nolang_dataset_name = '/content/size_comp_train_nolang'
datasets = load_dataset('json', data_files='/content/age_comp_train.jsonl').shuffle()
# nolang_dataset_test = load_dataset('json', data_files='/content/nolang_size_comp_dev.jsonl').shuffle()
# datasets = load_dataset('json', data_files='/content/nolang_breakable_train.jsonl').shuffle()
dataset_test = load_dataset('json', data_files='/content/age_comp_dev.jsonl').shuffle()

# # num
# datasets = load_dataset('json', data_files='/content/ant-syn_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/ant-syn_dev.jsonl').shuffle()

# age
# datasets = load_dataset('json', data_files='/content/age_comp_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/age_comp_dev.jsonl').shuffle()

datasets = datasets['train'].train_test_split(test_size=0.1)
datasets['validation'] = datasets['test']
datasets['test'] = dataset_test['train']

In [ ]:
datasets

# Common

In [ ]:

# @title model checkpoints
# eng_model_checkpoint = "bert-base-uncased"
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
model_checkpoint = 'bert-base-multilingual-cased'

# model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'

# roberta-fa-zwnj-base

# TODO: change Model to persian one


In [ ]:
# @title learning curve functions
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW


def create_dataloader(dataset, batch_size=BATCH_SIZE):
  dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = RandomSampler( dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
  return dataloader

def prepare_model(model_class, model_checkpoint):
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  base_model = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)
  
  model = model_class(base_model, tokenizer)
  model.cuda()
  return model

  

def learning_curve(model_class, model_checkpoint, test_dataset, dataset_name, learning_sizes = [64, 128, 256, 512, 1024, 2048, 4096], batch_size=BATCH_SIZE):
  # train_dataset = datasets['train']
  # val_dataset = datasets['validation']
  # val_dataset = prepare_dataset(val_dataset)
  test_dataset = test_dataset.map(fix_labels)
  test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label', 'options'])
  # validation_dataloader = create_dataloader(val_dataset, batch_size)
  test_dataloader = create_dataloader(test_dataset, batch_size)

  results = {}
  model = prepare_model(model_class, model_checkpoint)
  eval_result = evaluate_model(model, test_dataloader)
  results[0] = eval_result
  
  for c in learning_sizes:
    train_dataset2, val_dataset2 = prepare_dataset(dataset_name, c)
    train_dataloader = create_dataloader(train_dataset2, batch_size)
    val_dataloader = create_dataloader(val_dataset2, batch_size)
    model = prepare_model(model_class, model_checkpoint)
    train_model(model, train_dataloader, val_dataloader)
    eval_result = evaluate_model(model, test_dataloader)
    results[c] = eval_result
  return results


In [ ]:
# @title fix label

def fix_labels(ex):
  c=ex['question']['choices']
  options = []
  for i in c:
    options.append(i['text'])
  options = [tokenizer.encode(option)[1] for option in options]
  ex['fixed_label']=ex['label']
  # print(options)
  ex['options'] = torch.tensor(options)
  ex['question']['stem']=ex['question']['stem'].replace('[MASK]',tokenizer.mask_token)
  enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
  ex={**ex,**enc}
  sol = {}
  cols = ['input_ids', 'attention_mask', 'fixed_label', 'options']
  for k in ex:
    if k in cols:
      sol[k] = ex[k]
  return sol

def prepare_dataset(file_name, data_size = -1):
  # print(encoded_dataset[0])
  # raise 1
  # t = [0,0]
  # for m in encoded_dataset:
  #   t[m['fixed_label']]+=1

  # print(t)
  
    # print(dataset)
  #   d1 = []
  #   i = 0
  #   while len(d1)<data_size/2:
  #     if encoded_dataset[i]['fixed_label'] == 1:
  #       d1.append(encoded_dataset[i])
  #     i+=1
  #   # print(d1)
  #   d0 = []
  #   i = 0
  #   while len(d0)<data_size/2:
  #     if encoded_dataset[i]['fixed_label'] == 0:
  #       d0.append(encoded_dataset[i])
  #     i+=1
  # d0.extend(d1)
  # encoded_dataset = load_dataset('list',d0)

  if data_size != -1:
    file_name = file_name + '_' + str(data_size) + '.jsonl'
    datasets = load_dataset('json', data_files=file_name).shuffle()

    datasets = datasets['train'].train_test_split(test_size=0.1)
    datasets['validation'] = datasets['test']
  else:    
    file_name = file_name +  '.jsonl'
    datasets = load_dataset('json', data_files=file_name).shuffle()
    datasets = datasets['train'].train_test_split(test_size=0.1)
    # dataset = dataset.train_test_split(data_size, seed = np.random.randint(1000))['test']
  val_encoded_dataset = datasets['test'].map(fix_labels)
  train_encoded_dataset = datasets['train'].map(fix_labels)
  # encoded_dataset = encoded_dataset.shuffle()
  val_encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label', 'options'])
  train_encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label', 'options'])
  return train_encoded_dataset, val_encoded_dataset

# prepare_dataset(datasets['test'],64)

In [ ]:
# @title train_model function - real

def train_model(model, train_dataloader, validation_dataloader):
  print("=======================================================================")

  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []

  # Measure the total training time for the whole run.
  total_t0 = time.time()

  optimizer = AdamW(model.parameters(),
                    lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )
  total_steps = len(train_dataloader) * epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
  criterion = torch.nn.CrossEntropyLoss() 

  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 

          b_input_ids =batch['input_ids'].to(device)
          # print(b_input_ids)
          b_input_mask = batch['attention_mask'].to(device)
          # # token_type_ids = torch.stack(batch['token_type_ids']).to(device)
          b_labels = batch['fixed_label'].to(device)
          b_options = batch['options'].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # In PyTorch, calling `model` will in turn call the model's `forward` 
          # function and pass down the arguments. The `forward` function is 
          # documented here: 
          # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
          # The results are returned in a results object, documented here:
          # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
          # Specifically, we'll get the loss (because we provided labels) and the
          # "logits"--the model outputs prior to activation.
          # print(b_input_ids.shape,b_labels.shape,b_input_mask.shape)
          result = model(ids=b_input_ids, 
                        mask=b_input_mask, 
                        labels=b_labels,
                        options = b_options) 
          
          loss = criterion(result, b_labels)
          # print(loss)
          # loss = result.loss
          # logits = result.logits
          # print('-----------------loss------------')
          # print(loss)

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          # print(loss.item())
          total_train_loss += loss.item()


          # Perform a backward pass to calculate the gradients.
          # print(loss.requires_grad)
          # loss.requires_grad = True
          # loss = Variable(loss, requires_grad = True)
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          # raise 1

          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)
      # for p in params[0:5]:
      #     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
      # par = list(model.named_parameters())
      # params.append(par)
      # print(params[-4])
      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(training_time))
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()
      
      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using 
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          # print(batch)
          b_input_ids = batch['input_ids'].to(device)
          b_input_mask = batch['attention_mask'].to(device)
          b_labels = batch['fixed_label'].to(device)
          b_options = batch['options'].to(device)
          
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              result = model(ids=b_input_ids, 
                        mask=b_input_mask, 
                        labels=b_labels,
                        options = b_options)

          # Get the loss and "logits" output by the model. The "logits" are the 
          # output values prior to applying an activation function like the 
          # softmax.
          loss = criterion(result, b_labels)
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = result.detach().cpu().numpy()
          label_ids = b_labels.cpu().numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

  return avg_val_accuracy

In [ ]:
#  @title evaluate_model function

def evaluate_model(model, test_dataloader):
  model.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  # Evaluate data for one epoch
  for batch in test_dataloader:

      b_input_ids = batch['input_ids'].to(device)
      b_input_mask = batch['attention_mask'].to(device)
      b_labels = batch['fixed_label'].to(device)
      b_options= batch['options'].to(device)
      with torch.no_grad():
        result = model(ids=b_input_ids, 
                      mask=b_input_mask, 
                      labels=b_labels,
                      options=b_options)

      # Move logits and labels to CPU
      logits = result.detach().cpu().numpy()
      label_ids = b_labels.cpu().numpy()

      # Calculate the accuracy for this batch of test sentences, and
      # accumulate it over all batches.
      total_eval_accuracy += flat_accuracy(logits, label_ids)
      

  # Report the final accuracy for this validation run.
  avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
  print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))
  return avg_val_accuracy



In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:

# @title Compare1_MC_MLM
import torch
from torch.nn import functional


class Compare1_MC_MLM(torch.nn.Module):
  def __init__(self, model, tokenizer):
    # print("__________-----------------------______________")
    super(Compare1_MC_MLM, self).__init__()
    for p in model.bert.parameters():
      p.requires_grad=False
    self.model = model
    self.model.cuda()
    self.tokenizer = tokenizer
    # self.options = [self.tokenizer.encode(option)[1:3] for option in options]
    # print (self.options)
  
  def forward(self, ids, mask, labels, options):

    output = self.model(ids,attention_mask = mask)
    # options = [self.tokenizer.encode(option)[1] for option in options]
    logits = output.logits

    mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

    # softmax = functional.softmax(logits, dim = -1)
    # print(softmax.shape)
    mask1_word = []
    # mask2_word = []
    for i in range(0, len(mask_index)):
      mask1_word.append(logits[i][mask_index[i]])
    mask1_word =  torch.stack(mask1_word)

    indices1 = torch.tensor(options).to(device)

    predicted1 = []
    for i in range(len(mask1_word)):
      # print("mask, " , mask1_word[i])
      x = [mask1_word[i][j] for j in indices1[i]]
      x = torch.stack(x)
      # print(x.shape)
      predicted1.append(x)

    predicted1 = torch.stack(predicted1)

    predicted = functional.softmax(predicted1)

    return predicted


## curve

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

epochs=4
device = torch.device("cuda")
# model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
# model_checkpoint = 'bert-base-multilingual-cased'

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)
bertmodel.cuda()

# model = Compare1_MC_MLM(bertmodel,tokenizer)
# model.cuda()
# clear_output()
# train_model(model, train_dataloader, validation_dataloader)

In [ ]:
# @title trans_numbers,large_num2word functions

def num2word(num):
  # if num=='-':
  #   return 'منفی '
  num = int(num)
  result = ''
  word = {1:'یک',2:'دو',3:'سه',4:'چهار',5:'پنج',6:'شش',7:'هفت',8:'هشت',9:'نه',10:'ده',11:'یازده',12:'دوازده',13:'سیزده',14:'چهارده',15:'پانزده',16:'شانزده',17:'هفده',18:'هجده',19:'نوزده',20:'بیست',30:'سی',40:'چهل',50:'پنجاه',60:'شصت',70:'هفتاد',80:'هشتاد',90:'نود',100:'صد',200:'دویست',300:'سیصد',400:'چهارصد',500:'پانصد',600:'ششصد',700:'هفتصد',800:'هشتصد',900:'نهصد'}
  if num==0:
    return 'صفر'
  if num<0:
    result = 'منفی '
    num*=-1
  if num <= 20:
    result+= word[num]
  elif num<100:
    result+=word[10*int(num/10)]
    if num%10>0:
      result+=' و ' + word[num%10]
  elif num<1000:
    result+=word[100*int(num/100)]
    tmp = num%100
    if tmp>0:
      result+=' و '+ num2word(tmp)

  return result

def large_num2word(num):
  if num[0]=='-':
    num=num[1:]
    result='منفی '
  else:
    result=''
  suffix={0:'',1:' هزار',2:' ملیون',3:' میلیارد',4:' تریلیون',5:' تریلیارد'}
  triples = [num[0:len(num)%3]] if len(num)%3!=0 else []
  triples +=[num[i:i+3] for i in range(len(num)%3, len(num), 3)]
  triples.reverse()
  result +=num2word(triples[-1])+suffix[len(triples)-1]
  for i in range(len(triples)-2,-1,-1):
    n=num2word(triples[i])
    if n!='صفر':
      result+=' و '+n+suffix[i]
  return result

def trans_numbers(num):
  num = num.replace('1','۱')
  num = num.replace('2','۲')
  num = num.replace('3','۳')
  num = num.replace('4','۴')
  num = num.replace('5','۵')
  num = num.replace('6','۶')
  num = num.replace('7','۷')
  num = num.replace('8','۸')
  num = num.replace('9','۹')
  num = num.replace('0','۰')
  return num


In [ ]:
def predic_mask(text, option1, option2, model, tokenizer):
  op1 = tokenizer.encode_plus(option1, return_tensors = "pt")['input_ids'][0][1]
  op2 = tokenizer.encode_plus(option2,  return_tensors = "pt")['input_ids'][0][1]
  text  =text.replace('[MASK]', tokenizer.mask_token)
  input = tokenizer.encode_plus(text, return_tensors = "pt").to(device)
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
  output = model(**input)
  logits = output.logits
  # softmax = F.softmax(logits, dim = -1)
  mask_word = logits[0, mask_index, :]
  # k = tokenizer.vocab_size
  # topk = torch.topk(mask_word, k, dim = 1)
  p1 = mask_word[0][op1]
  p2 = mask_word[0][op2]
  if p1>p2:
    return option1
  else:
    return option2
  # mask_word_prob = topk[0][0]
  # mask_word = topk[1][0]
  # op1_index = torch.where(mask_word==op1)
  # op2_index = torch.where(mask_word==op2)
  # sent1 =  text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[op1_index]))
  # sent2 = text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[op2_index]))
  # if op1_index[0] > op2_index[0]:
  #   predicted = option2
  #   # print(sent2)
  # else:
  #   predicted = option1
  #   # print(sent1)
  # # print(predicted)
  # return predicted

In [ ]:
import torch 
from torch.nn import functional as F
# text = 'یک فرد num1 ساله [MASK] از یک فرد num2 ساله است.'
text = 'num1 [MASK] num2'
options = ['بیشتر','کمتر']
# predic_mask(text,'بزرگتر','کوچکتر', bertmodel,tokenizer)
eval = []
for i in range(15,46):
  eval.append([])
  for j in range(18,49):
    sent = text.replace('num1',trans_numbers(str(i)))
    sent = sent.replace('num2',trans_numbers(str(j)))
    p = predic_mask(sent,options[0],options[1],bertmodel,tokenizer)
    if p == options[0]:
      eval[i-15].append(0)
    elif p == options[1]:
      eval[i-15].append(1)
    else:
      print(p)
      raise 1

print(eval)

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt

ax = sns.heatmap(eval,vmin=0,vmax=1,cbar=False,xticklabels=range(18,49),yticklabels=range(15,46),square=True)#, linewidth=0.5)
from google.colab import files
# plt.show()
# plt.title('یک فرد ده ساله [جوان]‌تر از یک فرد بیست ساله است.')
plt.savefig("mbert_num_fa.png")
files.download("mbert_num_fa.png") 


In [ ]:
tokenizer.encode('بزرگتر')

In [ ]:
curve_info = learning_curve(Compare1_MC_MLM, model_checkpoint, test_dataset, dataset_name)


In [ ]:
nolang_curve_info = learning_curve(Compare1_MC_MLM, model_checkpoint, nolang_test_datasets, nolang_dataset_name)

In [ ]:
curve_info


In [ ]:
nolang_curve_info

In [ ]:
import matplotlib.pyplot as plt
# age alphabet nolang bert
# nolang_curve_info = {0: 0.37421875,10: 0.37421875,
#  64: 0.487890625,
#  128: 0.47890625,
#  256: 0.383203125,
#  512: 0.452734375,
#  1024: 0.457421875,
#  2048: 0.492578125,
#  4096: 0.6}

#  #age alphabet albert
# curve_info = {0:0.581640625,
#     10: 0.581640625,
#  64: 0.675,
#  128: 0.683984375,
#  256: 0.70546875,
#  512: 0.6175,
#  1024: 0.6703125,
#  2048: 0.621171875,
#  4096: 0.679296875}


plt.plot(list(curve_info.keys()), list(curve_info.values()), 'bo')
plt.plot(list(curve_info.keys()), list(curve_info.values()), 'b-',label = 'age comparison')

# plt.plot(list(nolang_curve_info.keys()), list(nolang_curve_info.values()), 'ro')
# plt.plot(list(nolang_curve_info.keys()), list(nolang_curve_info.values()), 'r--',label = 'age comparison nolang')

plt.xscale('log', basex=2)
plt.legend()
from google.colab import files
plt.savefig("num_mbert.png")
files.download("num_mbert.png") 

# Models 

In [ ]:

# @title Compare1_MC_MLM
import torch
from torch.nn import functional


class Compare1_MC_MLM(torch.nn.Module):
  def __init__(self, model, tokenizer):
    # print("__________-----------------------______________")
    super(Compare1_MC_MLM, self).__init__()
    for p in model.albert.parameters():
      p.requires_grad=False
    self.model = model
    self.model.cuda()
    self.tokenizer = tokenizer
    # self.options = [self.tokenizer.encode(option)[1:3] for option in options]
    # print (self.options)
  
  def forward(self, ids, mask, labels, options):

    output = self.model(ids,attention_mask = mask)
    # options = [self.tokenizer.encode(option)[1] for option in options]
    logits = output.logits

    mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

    # softmax = functional.softmax(logits, dim = -1)
    # print(softmax.shape)
    mask1_word = []
    # mask2_word = []
    for i in range(0, len(mask_index)):
      mask1_word.append(logits[i][mask_index[i]])
    mask1_word =  torch.stack(mask1_word)

    indices1 = torch.tensor(options).to(device)

    predicted1 = []
    for i in range(len(mask1_word)):
      # print("mask, " , mask1_word[i])
      x = [mask1_word[i][j] for j in indices1[i]]
      x = torch.stack(x)
      # print(x.shape)
      predicted1.append(x)

    predicted1 = torch.stack(predicted1)

    predicted = functional.softmax(predicted1)

    return predicted


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
from torch.nn import functional
import torch
model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
# model_checkpoint = 'bert-base-multilingual-cased'
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)

bertmodel.cuda()

In [ ]:
# for p in bertmodel.bert.parameters():
#   p.requires_grad=False


In [ ]:
# # @title fix label
# def fix_labels(ex):
#   c=ex['question']['choices']
#   # for i in c:
#   #   if i['label']==0:
#   #     if i['text']=='کوچکتر':
#   #       ex['fixed_label']=ex['label']
#   #     else:
#   #       ex['fixed_label']=1-ex['label']
#   ex['fixed_label']=ex['label']
  
#   ex['question']['stem']=ex['question']['stem'].replace('[MASK]', tokenizer.mask_token + ' ' + tokenizer.mask_token)
#   enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
  
#   ex={**ex,**enc}
#   return ex

# encoded_dataset = datasets.map(fix_labels)
# encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label'])
val_encoded_dataset = datasets['validation'].map(fix_labels)
train_encoded_dataset = datasets['train'].map(fix_labels)
# encoded_dataset = encoded_dataset.shuffle()
val_encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label', 'options'])
train_encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label', 'options'])
# return train_encoded_dataset, val_encoded_dataset


In [ ]:
# @title dataset loader
from torch.utils.data import TensorDataset, random_split

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# dataset = TensorDataset(input_ids, attention_masks, labels)

encoded_dataset = fix_labels(datasets)
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_encoded_dataset,  # The training samples.
            sampler = RandomSampler(train_encoded_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_encoded_dataset, # The validation samples.
            sampler = RandomSampler(val_encoded_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            encoded_dataset['test'], # The validation samples.
            sampler = SequentialSampler(encoded_dataset['test']), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
# for i , j in enumerate(test_dataloader):
#   print(i,j)
#   break

In [ ]:
for p in model.model.lm_head.parameters():
    print(p.requires_grad)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)
bertmodel.cuda()

model = Compare1_MC_MLM(bertmodel,tokenizer)
model.cuda()
clear_output()

In [ ]:
params = list(model.named_parameters())
# for p in params[0:5]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
for p in params:
  print(p[-1].grad)
  break
# params[-1][1].grad

In [ ]:
# print(loss.grad)

print(loss.retain_grad())
# # print(params[-1][1].grad)
# import gc

# gc.collect()
# torch.cuda.empty_cache()


In [ ]:
# result = model(ids=b_input_ids, 
#                 mask=b_input_mask, 
#                 labels=b_labels,
#                 options = b_options) 

loss = criterion(result, b_labels)
# # print(loss)
# # loss = result.loss
# # logits = result.logits
# # print('-----------------loss------------')
# # print(loss)

# # Accumulate the training loss over all of the batches so that we can
# # calculate the average loss at the end. `loss` is a Tensor containing a
# # single value; the `.item()` function just returns the Python value 
# # from the tensor.
# print(loss.item())
# # total_train_loss += loss.item()


# # Perform a backward pass to calculate the gradients.
# # print(loss.requires_grad)
# # loss.requires_grad = True
# # loss = Variable(loss, requires_grad = True)
# loss.backward()

# # Clip the norm of the gradients to 1.0.
# # This is to help prevent the "exploding gradients" problem.
# torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


### learning loop

In [ ]:
import random, time
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
criterion = torch.nn.CrossEntropyLoss() 

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        b_input_ids =batch['input_ids'].to(device)
        # print(b_input_ids)
        b_input_mask = batch['attention_mask'].to(device)
        # # token_type_ids = torch.stack(batch['token_type_ids']).to(device)
        b_labels = batch['fixed_label'].to(device)
        b_options = batch['options'].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        # print(b_input_ids.shape,b_labels.shape,b_input_mask.shape)
        result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels,
                       options = b_options) 
        
        loss = criterion(result, b_labels)
        # print(loss)
        # loss = result.loss
        # logits = result.logits
        # print('-----------------loss------------')
        # print(loss)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        # print(loss.item())
        total_train_loss += loss.item()


        # Perform a backward pass to calculate the gradients.
        # print(loss.requires_grad)
        # loss.requires_grad = True
        # loss = Variable(loss, requires_grad = True)
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        # raise 1

        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    # for p in params[0:5]:
    #     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    # par = list(model.named_parameters())
    # params.append(par)
    # print(params[-4])
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        # print(batch)
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['fixed_label'].to(device)
        b_options = batch['options'].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels,
                       options = b_options)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = criterion(result, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = result.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


In [ ]:
if params[0] != params[1]:
  print('1')
if params[0] != params[2]:
  print('2')
if params[0] != params[3]:
  print('3')

### evaluation

In [ ]:
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
# Evaluate data for one epoch
for batch in test_dataloader:

    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    b_labels = batch['fixed_label'].to(device)
    b_options = batch['options'].to(device)
    with torch.no_grad():
      result = model(ids=b_input_ids, 
                    mask=b_input_mask, 
                    labels=b_labels,
                     options = b_options)

    # Move logits and labels to CPU
    logits = result.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))



# TODO


*   pass tokenizer to fix label (it uses it)
*   List item



# Notes
## Run Results



In [ ]:
# size bert
curve = {64: 0.376953125,
 128: 0.35234375,
 256: 0.362109375,
 512: 0.48046875,
 1024: 0.4265625,
 2048: 0.487890625,
 4096: 0.545703125}
 
 # size nolang bert
 curve = {64: 0.507421875,
 128: 0.52421875,
 256: 0.5015625,
 512: 0.501171875,
 1024: 0.53671875,
 2048: 0.61171875,
 4096: 0.674609375}

# size albert
curve = {64: 0.47734375,
 128: 0.49296875,
 256: 0.49296875,
 512: 0.5203125,
 1024: 0.523046875,
 2048: 0.56171875,
 4096: 0.580078125}

 # size nolang albert
 curve = {64: 0.498046875,
 128: 0.48828125,
 256: 0.537109375,
 512: 0.545703125,
 1024: 0.576953125,
 2048: 0.559375,
 4096: 0.54765625}

 # age alphabet bert
curve_info = {0: 0.494140625,
 64: 0.4875,
 128: 0.358984375,
 256: 0.468359375,
 512: 0.38359375,
 1024: 0.460546875,
 2048: 0.3625,
 4096: 0.4}

# age alphabet nolang bert
nolang_curve_info = {0: 0.37421875,
 64: 0.487890625,
 128: 0.47890625,
 256: 0.383203125,
 512: 0.452734375,
 1024: 0.457421875,
 2048: 0.492578125,
 4096: 0.6}

 #age alphabet albert
 curve_info = {0: 0.581640625,
 64: 0.675,
 128: 0.683984375,
 256: 0.70546875,
 512: 0.5875,
 1024: 0.6703125,
 2048: 0.601171875,
 4096: 0.679296875}

 #age alphabet nolang albert
nolang_curve_info = {0: 0.49296875,
 64: 0.5875,
 128: 0.509375,
 256: 0.44296875,
 512: 0.509375,
 1024: 0.455859375,
 2048: 0.448046875,
 4096: 0.466796875}

#age mbert 
 curve_info = {0: 0.489453125,
 64: 0.49296875,
 128: 0.49296875,
 256: 0.49296875,
 512: 0.49296875,
 1024: 0.49609375,
 2048: 0.543359375,
 4096: 0.574609375}


# age mbert nolang
nolang_curve_info = {0: 0.409765625,
 64: 0.371484375,
 128: 0.503515625,
 256: 0.43203125,
 512: 0.441015625,
 1024: 0.509375,
 2048: 0.596875,
 4096: 0.64375}


#age alph mbert
 curve_info ={0: 0.489453125,
 64: 0.565625,
 128: 0.51328125,
 256: 0.602734375,
 512: 0.53359375,
 1024: 0.53046875,
 2048: 0.55859375,
 4096: 0.551171875}


# age alph mbert nolang
nolang_curve_info = {0: 0.409765625,
 64: 0.40859375,
 128: 0.38515625,
 256: 0.40859375,
 512: 0.4296875,
 1024: 0.505859375,
 2048: 0.599609375,
 4096: 0.64375}


 # size mbert
curve_info ={0: 0.494140625,
 64: 0.4953125,
 128: 0.4953125,
 256: 0.493359375,
 512: 0.498828125,
 1024: 0.49609375,
 2048: 0.54296875,
 4096: 0.496484375}


# size mbert nolang
nolang_curve_info = {0: 0.5453125,
 64: 0.54296875,
 128: 0.551953125,
 256: 0.523046875,
 512: 0.5125,
 1024: 0.55078125,
 2048: 0.475390625,
 4096: 0.4765625}

# num mbert 
curve_info = {0: 0.494140625,
 64: 0.46875,
 128: 0.4828125,
 256: 0.50078125,
 512: 0.4875,
 1024: 0.525,
 2048: 0.558203125,
 4096: 0.57578125}

# num mbert nolang
nolang_curve_info = {0: 0.510546875,
 64: 0.509375,
 128: 0.525,
 256: 0.5171875,
 512: 0.588671875,
 1024: 0.65546875,
 2048: 0.732421875,
 4096: 0.723828125}

In [ ]:

def diff(l1,l2):
  res = {}
  for k in l1:
    res[k] = max(0,l1[k]-l2[k])
  return res
    
def ws(l):
  w = [0.23, 0.2, 0.17, 0.14, 0.11, 0.08, 0.07]
  count = 0
  sum = 0
  i = 0
  for c, acc in l.items():
    sum += acc*w[i]
    count += w[i]
    i+=1
  return sum/count
